In [27]:
import json
import os
import csv
import re
import glob
import pandas as pd
from collections import Counter
from estnltk import Text
from estnltk.converters import json_to_text
from pandas import DataFrame


In [2]:
stopid_fail = 'estonian-stopwords.txt'
lemmad_fail = 'estonian-stopwords-lemmas.txt'

stopid_s = []
with open(stopid_fail, 'r') as sf:
    for line in sf:
        stopid_s.append(line.strip())

stopid_l = []
with open(lemmad_fail, 'r') as lf:
    for line in lf:
        stopid_l.append(line.strip())

In [3]:
def speech_jsonist(dir):

    sekundid = 0
    koik_konet = []
    
    for fail in os.listdir(dir):
        f_nimi = os.path.join(dir, fail)
        if not fail.endswith('.json'):
            continue
        with open(f_nimi, 'r') as f:
       
            andmed = json.load(f)
     
            for section in andmed['sections']:
                if 'turns' in section.keys():
                    algus = float(section['start'])
                    lopp = float(section['end'])
                    aeg_kokku = lopp - algus
                    sekundid += aeg_kokku

                    for turn in section['turns']:
                        kone = turn['transcript']
                        koik_konet.append(kone)

    return sekundid/60, ' '.join(koik_konet)

In [4]:
def loendamine(kone):
    
    
    koik_sonad = []
    koik_lemmad = []
    
    tekst = Text(kone).tag_layer('morph_analysis')
    for sone in tekst.morph_analysis:
        lemma = sone.lemma[0].lower()
        soneliik = sone.partofspeech[0]
                        
        if soneliik != 'Z':
            koik_lemmad.append(lemma)
            koik_sonad.append(sone.text.lower())
            
    return koik_sonad, koik_lemmad

In [5]:
def sagedus_loend(lemmad, stop_sonad):
    freq_counter = Counter()
    for lemma in lemmad:
        if lemma not in stop_sonad:
            freq_counter[lemma] += 1

    return freq_counter

In [6]:
minutid_mehed, kone_mehed = speech_jsonist('mehed')
print(minutid_mehed)

2590.5986666666695


In [7]:
minutid_naised, kone_naised = speech_jsonist('naised')
print(minutid_naised)

2010.3988333333323


In [8]:
print(minutid_mehed/60)

43.17664444444449


In [9]:
print(minutid_naised/60)

33.506647222222206


In [10]:
koik_sonad_mehed, koik_lemmad_mehed = loendamine(kone_mehed)

In [11]:
koik_sonad_naised, koik_lemmad_naised = loendamine(kone_naised)

Sagedusloendid

In [12]:
def sageduse_fail(failinimi, loend, minutid):
    with open(failinimi, 'w', encoding='utf-8', newline='') as out_f:
        writer = csv.writer(out_f, delimiter=';')
    
  
        writer.writerow(['lemma', 'raw_count', 'freq_per_60_min'])

        for lemma, cnt in loend.most_common():
            freq_per_1000 = (cnt / minutid) * 60
            writer.writerow([lemma, cnt, f"{freq_per_1000:.3f}"])

In [13]:
filename_out = 'lemmade_sagedus_by_min_mehed_veel.csv'

sageduse_fail(filename_out, sagedus_loend(koik_lemmad_mehed, stopid_l), minutid_mehed)

In [14]:
filename_out = 'lemmade_sagedus_by_min_naised_veel.csv'

sageduse_fail(filename_out, sagedus_loend(koik_lemmad_naised, stopid_l), minutid_naised)

Bigrammid ja trigrammid:

In [15]:
def bi_ja_tri_grammid(lemmad, stop_lemmad, sonad, stop_sonad):

    bigram_counter_lemma = Counter()
    trigram_counter_lemma = Counter()
    
    for i in range(len(lemmad)-1):
        bigram = (lemmad[i].lower(), lemmad[i+1].lower())
        if bigram[0] != bigram[1]:
    
            if bigram[0] not in stop_lemmad and bigram[1] not in stop_lemmad:
                bigram_counter_lemma[bigram] += 1
            elif bigram[1] not in stop_lemmad:
                bigram_counter_lemma[bigram] += 1
    
    for i in range(len(lemmad) - 2):
        w1 = lemmad[i].lower()
        w2 = lemmad[i+1].lower()
        w3 = lemmad[i+2].lower()
        

        if w1 != w2 and w2 != w3 and w1 != w3:
            if w1 not in stop_lemmad and w2 not in stop_lemmad and w3 not in stop_lemmad:
                trigram = (w1, w2, w3)
                trigram_counter_lemma[trigram] += 1
            elif w1 not in stop_lemmad and  w3 not in stop_lemmad:
                trigram = (w1, w2, w3)
                trigram_counter_lemma[trigram] += 1
    
    print("Top 10 bigrams (lemmas):")
    for bg, cnt in bigram_counter_lemma.most_common(10):
        print(bg, cnt)
        
    print("\nTop 10 trigrams (lemmas):")
    for tg, cnt in trigram_counter_lemma.most_common(10):
        print(tg, cnt)
    

    bigram_counter_raw = Counter()
    trigram_counter_raw = Counter()
    
    for i in range(len(sonad)-1):
        bigram = (sonad[i].lower(), sonad[i+1].lower())
        if bigram[0] != bigram[1]:
            if bigram[0] not in stop_sonad and bigram[1] not in stop_sonad:
                bigram_counter_raw[bigram] += 1
            elif bigram[1] not in stop_sonad:
                bigram_counter_raw[bigram] += 1
    
    for i in range(len(sonad) - 2):
        w1 = sonad[i].lower()
        w2 = sonad[i+1].lower()
        w3 = sonad[i+2].lower()
        

        if w1 != w2 and w2 != w3 and w1 != w3:
            if w1 not in stop_sonad and w2 not in stop_sonad and w3 not in stop_sonad:
                trigram = (w1, w2, w3)
                trigram_counter_raw[trigram] += 1
            elif w1 not in stop_sonad and  w3 not in stop_sonad:
                trigram = (w1, w2, w3)
                trigram_counter_raw[trigram] += 1
    
    print("\nTop 10 bigrams (raw forms):")
    for bg, cnt in bigram_counter_raw.most_common(10):
        print(bg, cnt)
        
    print("\nTop 10 trigrams (raw forms):")
    for tg, cnt in trigram_counter_raw.most_common(10):
        print(tg, cnt)
    return bigram_counter_lemma, trigram_counter_lemma, bigram_counter_raw, trigram_counter_raw

In [16]:
bigrammid_lem_mehed, trigrammid_lem_mehed, bigrammid_raw_mehed, trigrammid_raw_mehed = bi_ja_tri_grammid(koik_lemmad_mehed, 
                                                                                                         stopid_l, koik_sonad_mehed, stopid_s)

Top 10 bigrams (lemmas):
('ei', 'teadma') 854
('mina', 'arvama') 656
('see', 'mõte') 531
('mina', 'ütlema') 346
('see', 'asi') 302
('saama', 'aru') 298
('mina', 'tahtma') 289
('mina', 'arust') 273
('kogu', 'aeg') 236
('mina', 'rääkima') 182

Top 10 trigrams (lemmas):
('eesti', 'puue', 'inimene') 28
('puue', 'inimene', 'koda') 27
('mees', 'ei', 'nutma') 20
('no', 'see', 'mõte') 17
('teadma', 'mis', 'asi') 13
('no', 'mina', 'ütlema') 12
('jutt', 'roheline', 'stuudio') 10
('ütlema', 'et', 'kuulma') 10
('tõde', 'ja', 'õigus') 9
('no', 'mina', 'arvama') 9

Top 10 bigrams (raw forms):
('ei', 'tea') 793
('ma', 'arvan') 597
('selles', 'mõttes') 467
('minu', 'arust') 272
('kogu', 'aeg') 234
('ma', 'ütlen') 145
('see', 'ongi') 127
('väga', 'hea') 122
('ma', 'tean') 122
('saan', 'aru') 118

Top 10 trigrams (raw forms):
('eesti', 'puuetega', 'inimeste') 22
('mehed', 'ei', 'nuta') 18
('no', 'selles', 'mõttes') 17
('jutud', 'rohelises', 'stuudios') 10
('no', 'ma', 'arvan') 9
('puuetega', 'inimeste',

In [17]:
bigrammid_lem_naised, trigrammid_lem_naised, bigrammid_raw_naised, trigrammid_raw_naised = bi_ja_tri_grammid(koik_lemmad_naised, 
                                                                                                             stopid_l, koik_sonad_naised, stopid_s)

Top 10 bigrams (lemmas):
('ei', 'teadma') 446
('mina', 'arvama') 374
('see', 'mõte') 300
('mina', 'tahtma') 291
('olema', 'tegelikult') 229
('mina', 'ütlema') 226
('mina', 'rääkima') 212
('see', 'inimene') 209
('olema', 'hästi') 201
('olema', 'vaja') 180

Top 10 trigrams (lemmas):
('tartu', 'noorsootöö', 'keskus') 19
('mees', 'ja', 'naine') 13
('noor', 'vaimne', 'tervis') 11
('ütlema', 'et', 'kuulma') 10
('hästi', 'palju', 'erinev') 9
('ema', 'ja', 'isa') 8
('teinekord', 'ikka', 'jälle') 8
('no', 'mina', 'arvama') 8
('no', 'mina', 'ütlema') 8
('puudutama', 'mina', 'saade') 7

Top 10 bigrams (raw forms):
('ei', 'tea') 415
('ma', 'arvan') 337
('selles', 'mõttes') 274
('et', 'tegelikult') 173
('kogu', 'aeg') 159
('on', 'tegelikult') 158
('on', 'hästi') 144
('ma', 'tahan') 136
('on', 'vaja') 128
('me', 'räägime') 117

Top 10 trigrams (raw forms):
('tartu', 'noorsootöö', 'keskuse') 10
('vaatasite', 'vaadake', 'teinekord') 9
('teinekord', 'ikka', 'jälle') 8
('no', 'ma', 'arvan') 8
('hästi', 

In [18]:
def bi_ja_tri_failid(filename_bigrams_lemma, filename_trigrams_lemma, filename_bigrams_raw, filename_trigrams_raw, 
                    bigram_counter_lemma, trigram_counter_lemma, bigram_counter_raw, trigram_counter_raw, total_minutes):
   
    with open(filename_bigrams_lemma, 'w', encoding='utf-8', newline='') as out_f:
        writer = csv.writer(out_f, delimiter=';')

        writer.writerow(['ngram', 'count', 'freq_per_60_min'])
        
        for bg, cnt in bigram_counter_lemma.most_common():

            freq_1000 = (cnt / total_minutes) * 60  
            

            bg_str = ' '.join(bg)
            

            writer.writerow([bg_str, cnt, f"{freq_1000:.4f}"])
    
    with open(filename_trigrams_lemma, 'w', encoding='utf-8', newline='') as out_f:
        writer = csv.writer(out_f, delimiter=';')

        writer.writerow(['ngram', 'count', 'freq_per_60_min'])
        
        for tg, cnt in trigram_counter_lemma.most_common():

            freq_1000 = (cnt / total_minutes) * 60  

            tg_str = ' '.join(tg)
            

            writer.writerow([tg_str, cnt, f"{freq_1000:.4f}"])
    
    with open(filename_bigrams_raw, 'w', encoding='utf-8', newline='') as out_f:
        writer = csv.writer(out_f, delimiter=';')

        writer.writerow(['ngram', 'count', 'freq_per_60_min'])
        
        for bg, cnt in bigram_counter_raw.most_common():

            freq_1000 = (cnt / total_minutes) * 60  
            

            bg_str = ' '.join(bg)
            

            writer.writerow([bg_str, cnt, f"{freq_1000:.4f}"])
    
    with open(filename_trigrams_raw, 'w', encoding='utf-8', newline='') as out_f:
        writer = csv.writer(out_f, delimiter=';')
 
        writer.writerow(['ngram', 'count', 'freq_per_60_min'])
        
        for tg, cnt in trigram_counter_raw.most_common():
            freq_1000 = (cnt / total_minutes) * 60 
            

            tg_str = ' '.join(tg)
            
            writer.writerow([tg_str, cnt, f"{freq_1000:.4f}"])


In [19]:
filename_bigrams_lemma_mehed = 'n_grammid/bigrams_lemma_per_60_minutes_mehed.csv'
filename_trigrams_lemma_mehed = 'n_grammid/trigrams_lemma_per_60_minutes_mehed.csv'
filename_bigrams_raw_mehed = 'n_grammid/bigrams_raw_per_60_minutes_mehed.csv'
filename_trigrams_raw_mehed = 'n_grammid/trigrams_raw_per_60_minutes_mehed.csv'

filename_bigrams_lemma_naised = 'n_grammid/bigrams_lemma_per_60_minutes_naised.csv'
filename_trigrams_lemma_naised = 'n_grammid/trigrams_lemma_per_60_minutes_naised.csv'
filename_bigrams_raw_naised = 'n_grammid/bigrams_raw_per_60_minutes_naised.csv'
filename_trigrams_raw_naised = 'n_grammid/trigrams_raw_per_60_minutes_naised.csv'

In [20]:
bi_ja_tri_failid(filename_bigrams_lemma_mehed, filename_trigrams_lemma_mehed, filename_bigrams_raw_mehed, filename_trigrams_raw_mehed, 
                    bigrammid_lem_mehed, trigrammid_lem_mehed, bigrammid_raw_mehed, trigrammid_raw_mehed, minutid_mehed)

In [21]:
bi_ja_tri_failid(filename_bigrams_lemma_naised, filename_trigrams_lemma_naised, filename_bigrams_raw_naised, filename_trigrams_raw_naised, 
                    bigrammid_lem_naised, trigrammid_lem_naised, bigrammid_raw_naised, trigrammid_raw_naised, minutid_naised)

Sõnaliigid:

In [22]:
def speech_sections(dir_path, gender_tag):
    records = []
    for fname in os.listdir(dir_path):
        if not fname.endswith('.json'):
            continue
        path = os.path.join(dir_path, fname)
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        episode_id = fname.replace('.json', '')
        for idx, section in enumerate(data.get('sections', [])):
            turns = section.get('turns')
            if not turns:
                continue
            start, end = float(section['start']), float(section['end'])
            minutes = (end - start) / 60.0
            transcript = ' '.join(t['transcript'] for t in turns)
            records.append({
                'episode_id': episode_id,
                'section_index': idx,
                'gender': gender_tag,
                'minutes': minutes,
                'transcript': transcript
            })
    return records

In [23]:
def count_pos(text_str, skip_lemmas):
    txt = Text(text_str)
    txt.tag_layer('morph_analysis')
    counters = {
        'omadussõnad': Counter(),
        'nimisõnad':   Counter(),
        'adverbid':    Counter(),
        'verbid':      Counter(),
        'asesõnad':    Counter(),
        'kaassõnad':   Counter(),
    }
    tag_map = {
        'A': 'omadussõnad',
        'S': 'nimisõnad',
        'D': 'adverbid',
        'V': 'verbid',
        'P': 'asesõnad',
        'K': 'kaassõnad'
    }

    for token in txt.morph_analysis:
        tag = token.partofspeech[0]
        if tag == 'V' and token.lemma[0].lower() in skip_lemmas:
            continue
        if tag in tag_map:
            key = tag_map[tag]
            form = token.lemma[0].lower()
            counters[key][form] += 1
    return counters

In [24]:
men_sections = speech_sections('mehed', 'M')
women_sections = speech_sections('naised', 'F')
all_sections = men_sections + women_sections

In [28]:
eituse_partiklid = ['ei', 'ära']
# DataFrame
records = []
for sec in all_sections:

    counts = count_pos(sec['transcript'], eituse_partiklid)
    for pos, counter in counts.items():
        for form, cnt in counter.items():
            freq = (cnt / sec['minutes']) * 60 if sec['minutes'] > 0 else 0.0
            records.append({
                'episode_id': sec['episode_id'],
                'section_index': sec['section_index'],
                'gender': sec['gender'],
                'minutes': sec['minutes'],
                'sõnaliik': pos,
                'feature': form,
                'raw_count': cnt,
                'freq_per_60_min': freq
            })

df_sections_pos = pd.DataFrame(records)

# CSV 
os.makedirs('pos_sections_csv', exist_ok=True)
for pos in df_sections_pos['sõnaliik'].unique():
    df_subset = df_sections_pos[df_sections_pos['sõnaliik'] == pos]
    filename = f'pos_sections_csv/{pos}_by_section.csv'
    df_subset.to_csv(filename, index=False)
    print(f"Salvestatud {len(df_subset)} read - {filename}")

Saved 15372 rows to pos_sections_csv/omadussõnad_by_section.csv
Saved 58898 rows to pos_sections_csv/nimisõnad_by_section.csv
Saved 35801 rows to pos_sections_csv/adverbid_by_section.csv
Saved 29373 rows to pos_sections_csv/verbid_by_section.csv
Saved 8888 rows to pos_sections_csv/asesõnad_by_section.csv
Saved 5426 rows to pos_sections_csv/kaassõnad_by_section.csv


In [29]:
input_dir = 'pos_sections_csv'
output_dir = 'pos_sections_csv/pos_sections_counter'
os.makedirs(output_dir, exist_ok=True)


for filepath in glob.glob(os.path.join(input_dir, '*.csv')):
    df = pd.read_csv(filepath)
    
    grouped = (
        df
        .drop(columns=['feature'])
        .groupby(['episode_id', 'section_index', 'gender', 'minutes', 'sõnaliik'], as_index=False)
        .agg({
            'raw_count': 'sum',
            'freq_per_60_min': 'sum'
        })
    )
    
    filename = os.path.basename(filepath)
    output_path = os.path.join(output_dir, filename)
    grouped.to_csv(output_path, index=False)

    print(f'{filename} → {output_path}')

kaassõnad_by_section.csv → pos_sections_csv/pos_sections_counter/kaassõnad_by_section.csv
omadussõnad_by_section.csv → pos_sections_csv/pos_sections_counter/omadussõnad_by_section.csv
verbid_by_section.csv → pos_sections_csv/pos_sections_counter/verbid_by_section.csv
adverbid_by_section.csv → pos_sections_csv/pos_sections_counter/adverbid_by_section.csv
nimisõnad_by_section.csv → pos_sections_csv/pos_sections_counter/nimisõnad_by_section.csv
asesõnad_by_section.csv → pos_sections_csv/pos_sections_counter/asesõnad_by_section.csv


In [30]:
eituse_partiklid = ['ei', 'ära']
es_ja_te_pronoomenid = ['mina', 'sina']
tingiv_koneviis = ['ks', 'ksin', 'ksid', 'ksime', 'ksite', 'taks', 'nuksin', 'nuksid', 'nuks', 'nuksime', 'nuksite', 'tuks']
umbisikuline = ['ti', 'takse', 'ti', 'tud', 'taks', 'tuks', 'tagu']
kaskiv_koneviis = ['o', 'ge', 'gem']
kp_markerid = ['arvan', 'usun', 'kardan', 'tundub', 'paistab', 'näib', 'loodan', 'ütleme']
verbid_partiklidena = ['ootama', 'vaatama', 'kuulama', 'kuulma']

intensiivistajad_loendist = []
with open('intensiivistajad.txt', 'r', encoding='utf-8') as f:
    for line in f:
        intensiivistajad_loendist.append(line.strip())

In [32]:
eitused = []
esimese_ja_teise_isik = []
intensiivistajad_omadussonadega = []
intensiivistajad_verbidega = []
intensiivistajad_adverbidega = []
tingiv_koneviis_loend = []
umbisikuline_loend = []
kaskiv_koneviis_loend = []
kp_markerid_loend = []
verbid_partiklidena_loend = []


for sec in all_sections:
    plok = sec['transcript']

    plok_nltk = Text(plok).tag_layer('morph_analysis')
    for lause in plok_nltk.sentences:      
        for i, sone in enumerate(lause):
            if sone.lemma[0] in eituse_partiklid and lause[i+1].partofspeech[0] == 'V':
                eitused.append({
                    'episode_id': sec['episode_id'],
                    'section_index': sec['section_index'],
                    'gender': sec['gender'],
                    'minutes': sec['minutes'],
                    'eituse': sone.lemma[0],
                    'verb': lause[i+1].lemma[0],
                    'lause': " ".join([sone.text for sone in lause])
                })
                
            if sone.lemma[0] in es_ja_te_pronoomenid:
                esimese_ja_teise_isik.append({
                    'episode_id': sec['episode_id'],
                    'section_index': sec['section_index'],
                    'gender': sec['gender'],
                    'minutes': sec['minutes'],
                    'pronoomen': sone.text.lower(),
                    'lause': " ".join([sone.text for sone in lause])
                })
                
            if sone.lemma[0] in intensiivistajad_loendist:
                if lause[i+1].partofspeech[0] == 'A':
                    intensiivistajad_omadussonadega.append({
                        'episode_id': sec['episode_id'],
                        'section_index': sec['section_index'],
                        'gender': sec['gender'],
                        'minutes': sec['minutes'],
                        'pos': lause[i+1].partofspeech[0],
                        'inetnsiivistaja': sone.lemma[0],
                        'sona': lause[i+1].lemma[0],
                        'lause': " ".join([sone.text for sone in lause])
                    })
                    
                elif lause[i+1].partofspeech[0] == 'V':
                    intensiivistajad_verbidega.append({
                        'episode_id': sec['episode_id'],
                        'section_index': sec['section_index'],
                        'gender': sec['gender'],
                        'minutes': sec['minutes'],
                        'pos': lause[i+1].partofspeech[0],
                        'inetnsiivistaja': sone.lemma[0],
                        'sona': lause[i+1].lemma[0],
                        'lause': " ".join([sone.text for sone in lause])
                    })
                elif lause[i+1].partofspeech[0] == 'D':
                    intensiivistajad_adverbidega.append({
                        'episode_id': sec['episode_id'],
                        'section_index': sec['section_index'],
                        'gender': sec['gender'],
                        'minutes': sec['minutes'],
                        'pos': lause[i+1].partofspeech[0],
                        'inetnsiivistaja': sone.lemma[0],
                        'sona': lause[i+1].lemma[0],
                        'lause': " ".join([sone.text for sone in lause])
                    })

            if sone.partofspeech[0] == 'V':
                if sone.text in kp_markerid:
                    kp_markerid_loend.append({
                        'episode_id': sec['episode_id'],
                        'section_index': sec['section_index'],
                        'gender': sec['gender'],
                        'minutes': sec['minutes'],
                        'verb': sone.text,
                        'lause': " ".join([sone.text for sone in lause])
                     })
                if sone.form[0] in tingiv_koneviis:
                    tingiv_koneviis_loend.append({
                        'episode_id': sec['episode_id'],
                        'section_index': sec['section_index'],
                        'gender': sec['gender'],
                        'minutes': sec['minutes'],
                        'verb': sone.text,
                        'lause': " ".join([sone.text for sone in lause])
                    })
                    
                elif sone.form[0] in umbisikuline:
                    umbisikuline_loend.append({
                        'episode_id': sec['episode_id'],
                        'section_index': sec['section_index'],
                        'gender': sec['gender'],
                        'minutes': sec['minutes'],
                        'verb': sone.text,
                        'lause': " ".join([sone.text for sone in lause])
                    })
                elif sone.form[0] in kaskiv_koneviis and lause[i-1].text.lower() not in ['ei', 'et']:
                    if sone.lemma[0] in verbid_partiklidena:
                        verbid_partiklidena_loend.append({
                            'episode_id': sec['episode_id'],
                            'section_index': sec['section_index'],
                            'gender': sec['gender'],
                            'minutes': sec['minutes'],
                            'verb': sone.text,
                            'lause': " ".join([sone.text for sone in lause])
                        })
                    else:
                        kaskiv_koneviis_loend.append({
                            'episode_id': sec['episode_id'],
                            'section_index': sec['section_index'],
                            'gender': sec['gender'],
                            'minutes': sec['minutes'],
                            'verb': sone.text,
                            'lause': " ".join([sone.text for sone in lause])
                         })

df_eitused = pd.DataFrame(eitused)
df_esimese_ja_teise_isik = pd.DataFrame(esimese_ja_teise_isik)
df_intensiivistajad_omadussonadega = pd.DataFrame(intensiivistajad_omadussonadega)
df_intensiivistajad_verbidega = pd.DataFrame(intensiivistajad_verbidega)
df_intensiivistajad_adverbidega = pd.DataFrame(intensiivistajad_adverbidega)
df_tingiv = pd.DataFrame(tingiv_koneviis_loend)
df_umbisikuline = pd.DataFrame(umbisikuline_loend)
df_kaskiv = pd.DataFrame(kaskiv_koneviis_loend)
df_kp = pd.DataFrame(kp_markerid_loend)
df_verbid_partiklidena = pd.DataFrame(verbid_partiklidena_loend)

df_eitused.to_csv('tabelid_kontrollimiseks/eitused.csv', index=False)
df_esimese_ja_teise_isik.to_csv('tabelid_kontrollimiseks/esimese_ja_teise_isik.csv', index=False)
df_intensiivistajad_omadussonadega.to_csv('tabelid_kontrollimiseks/intensiivistajad_omdasusonadega.csv', index=False)
df_intensiivistajad_verbidega.to_csv('tabelid_kontrollimiseks/intensiivistajad_verbidega.csv', index=False)
df_intensiivistajad_adverbidega.to_csv('tabelid_kontrollimiseks/intensiivistajad_adverbidega.csv', index=False)
df_tingiv.to_csv('tabelid_kontrollimiseks/tingiv.csv', index=False)
df_umbisikuline.to_csv('tabelid_kontrollimiseks/umbisikuline.csv', index=False)
df_kaskiv.to_csv('tabelid_kontrollimiseks/kaskiv.csv', index=False)
df_kp.to_csv('tabelid_kontrollimiseks/kp_markerid.csv', index=False)
df_verbid_partiklidena.to_csv('tabelid_kontrollimiseks/vaata_oota.csv', index=False)

In [33]:
partiklid = []
for sec in all_sections:
    plok = sec['transcript']

    plok_nltk = Text(plok).tag_layer('morph_analysis')
    for lause in plok_nltk.sentences:      
        for i, sone in enumerate(lause):
            if sone.partofspeech[0] == 'I' or sone.lemma[0] == 'nagu':
                if sone.lemma[0] == 'tere':
                    continue

                
                if i-2 < 0 or i+2 >= len(lause):
                    continue

            
                if (lause[i-2].text != sone.text) or (lause[i+2].text != sone.text):
                    partiklid.append({
                        'episode_id': sec['episode_id'],
                        'section_index': sec['section_index'],
                        'gender': sec['gender'],
                        'minutes': sec['minutes'],
                        'partikkel': sone.lemma[0],
                        'lause': " ".join([tok.text for tok in lause])
                    })

df_partiklid = pd.DataFrame(partiklid)  
df_partiklid.to_csv('tabelid_kontrollimiseks/partiklid.csv', index=False)
                #print(lause.text)

In [34]:
df = pd.read_csv('tabelid_kontrollimiseks/vaata_oota.csv')


particles = ['no', 'noh', 'nagu', 'aga', 'siis']
particle_pat = r'(?:' + '|'.join(particles) + r')'  # (?:no|noh|nagu|aga|siis)

def classify(row):
    verb = re.escape(row['verb'])
    text = row['lause']

    # verbi rea alguses, millele järgneb koma
    start_pattern = rf'(?i)^\s*{verb}\b\s*,'
    if re.match(start_pattern, text):
        return 1

    # või
    any_pattern = (
        rf'(?i)'                    
        rf'\b'                          # sõna piir
        rf'(?:{particle_pat}\s+)?'      # partikkel enne
        rf'{verb}\b'                    # verb
        rf'(?:\s+{particle_pat})?'      # partikkel pärast
        rf'\s*,'                        # tühikud (kui on), koma
    )
    return 1 if re.search(any_pattern, text) else 0

df['mood_flag'] = df.apply(classify, axis=1)
df.to_csv('tabelid_kontrollimiseks/verbid_partiklidena_kontrollimiseks.csv', index=False)


Kui tabelid on kontrollitud

In [35]:
input_path  = "tabelid_kontrollimiseks/verbid_partiklidena_kontrollitud.csv"
output_path = "tabelid_kontrollimiseks/verbid_partiklidena_kontrollitud_uus.csv"

df = pd.read_csv(input_path, sep=";")
df["mood_flag"] = pd.to_numeric(df["mood_flag"], errors="coerce")
df_clean = df[df["mood_flag"] != 0]
df_clean.to_csv(output_path, sep=";", index=False)

print(f"Salvestatud {output_path}")


Salvestatud tabelid_kontrollimiseks/verbid_partiklidena_kontrollitud_uus.csv


In [39]:
import pandas as pd

def aggregate_section_counts(
    input_path: str,
    output_path: str,
    sep: str = ';',
    group_cols: list = None
) -> pd.DataFrame:
   
    df = pd.read_csv(input_path, sep=sep)
    
   
    if group_cols is None:
        group_cols = ['episode_id', 'section_index', 'gender', 'minutes']
    

    grouped = (
        df
        .groupby(group_cols, as_index=False)
        .size()
        .rename(columns={'size': 'absolut_freq'})
    )
    

    grouped['freq_per_hour'] = grouped['absolut_freq'] / (grouped['minutes'] / 60)
    

    grouped.to_csv(output_path, sep=sep, index=False)
    
    print(f"Salvestatud {output_path}")
    return grouped


#df_verbid_partiklidena = aggregate_section_counts(
  #   input_path='tabelid_kontrollimiseks/verbid_partiklidena_sections_uus.csv',
  #   output_path='tabelid_kontrollimiseks/sections/verbid_partiklidena_sections.csv',
  #   sep=';',
# )

df_esimese_ja_teise_isik = aggregate_section_counts(
     input_path='tabelid_kontrollimiseks/esimese_ja_teise_isik.csv',
     output_path='tabelid_kontrollimiseks/sections/esimese_ja_teise_isik_sections.csv',
     sep=',',
 )
df_partiklid = aggregate_section_counts(
     input_path='tabelid_kontrollimiseks/partiklid.csv',
     output_path='tabelid_kontrollimiseks/sections/partikkel_sections.csv',
     sep=',',
 )
df_kaskiv = aggregate_section_counts(
     input_path='tabelid_kontrollimiseks/kaskiv.csv',
     output_path='tabelid_kontrollimiseks/sections/kaskiv_sections.csv',
     sep=',',
 )
df_tingiv = aggregate_section_counts(
     input_path='tabelid_kontrollimiseks/tingiv.csv',
     output_path='tabelid_kontrollimiseks/sections/tingiv_sections.csv',
     sep=',',
 )
df_umbisikuline = aggregate_section_counts(
     input_path='tabelid_kontrollimiseks/umbisikuline.csv',
     output_path='tabelid_kontrollimiseks/sections/umbisikuline_sections.csv',
     sep=',',
 )
df_kp = aggregate_section_counts(
     input_path='tabelid_kontrollimiseks/kp_markerid.csv',
     output_path='tabelid_kontrollimiseks/sections/kp_markerid_sections.csv',
     sep=',',
 )
df_umbisikuline = aggregate_section_counts(
     input_path='tabelid_kontrollimiseks/umbisikuline.csv',
     output_path='tabelid_kontrollimiseks/sections/umbisikuline_sections.csv',
     sep=',',
 )
df_omadussonadega = aggregate_section_counts(
     input_path='tabelid_kontrollimiseks/intensiivistajad_omdasusonadega.csv',
     output_path='tabelid_kontrollimiseks/sections/intensiivistajad_omadussonadega_sections.csv',
     sep=',',
 )
df_verbidega = aggregate_section_counts(
     input_path='tabelid_kontrollimiseks/intensiivistajad_verbidega.csv',
     output_path='tabelid_kontrollimiseks/sections/intensiivistajad_verbidega_sections.csv',
     sep=',',
 )
df_adverbidega = aggregate_section_counts(
     input_path='tabelid_kontrollimiseks/intensiivistajad_adverbidega.csv',
     output_path='tabelid_kontrollimiseks/sections/intensiivistajad_adverbidega_sections.csv',
     sep=',',
 )

Salvestatud tabelid_kontrollimiseks/sections/esimese_ja_teise_isik_sections.csv
Salvestatud tabelid_kontrollimiseks/sections/partikkel_sections.csv
Salvestatud tabelid_kontrollimiseks/sections/kaskiv_sections.csv
Salvestatud tabelid_kontrollimiseks/sections/tingiv_sections.csv
Salvestatud tabelid_kontrollimiseks/sections/umbisikuline_sections.csv
Salvestatud tabelid_kontrollimiseks/sections/kp_markerid_sections.csv
Salvestatud tabelid_kontrollimiseks/sections/umbisikuline_sections.csv
Salvestatud tabelid_kontrollimiseks/sections/intensiivistajad_omadussonadega_sections.csv
Salvestatud tabelid_kontrollimiseks/sections/intensiivistajad_verbidega_sections.csv
Salvestatud tabelid_kontrollimiseks/sections/intensiivistajad_adverbidega_sections.csv


In [42]:
def save_tables_per_particle(
    csv_path: str,
    column_name: str,
    out_dir: str,
    threshold: int = None,
    delimiter: str = ',',
    encoding: str = 'utf-8'
):
    
    df = pd.read_csv(csv_path, sep=delimiter, encoding=encoding)


    os.makedirs(out_dir, exist_ok=True)


    for part_value, group in df.groupby(column_name):
        #
        if threshold is not None and len(group) <= threshold:
            continue


        safe = re.sub(r'[^\w\-]+', '_', str(part_value)).strip('_')
        filename = f"{safe}.csv"


        path = os.path.join(out_dir, filename)
        group.to_csv(path, index=False)
        print(f"Salvestatud {len(group)} read → {path}")


In [43]:
if __name__ == "__main__":
    save_tables_per_particle(
        csv_path='tabelid_kontrollimiseks/partiklid.csv',
        column_name='partikkel',
        out_dir='.../particles_per_value',
        threshold=50  
    )

Salvestatud 121 read → .../particles_per_value/aitäh.csv
Salvestatud 111 read → .../particles_per_value/jah.csv
Salvestatud 59 read → .../particles_per_value/kurat.csv
Salvestatud 10957 read → .../particles_per_value/nagu.csv
Salvestatud 669 read → .../particles_per_value/no.csv
Salvestatud 575 read → .../particles_per_value/noh.csv
Salvestatud 58 read → .../particles_per_value/oh.csv
Salvestatud 349 read → .../particles_per_value/okei.csv
Salvestatud 133 read → .../particles_per_value/vot.csv


In [48]:
def make_section_freq_tables(
    input_dir: str,
    output_dir: str,
    group_cols=('episode_id', 'section_index', 'gender', 'minutes'),
    input_sep=',',
    output_sep=';'
):

    os.makedirs(output_dir, exist_ok=True)
    pattern = os.path.join(input_dir, '*.csv')
    for in_path in glob.glob(pattern):

        df = pd.read_csv(in_path, sep=input_sep)
        
        grouped = (
            df
            .groupby(list(group_cols), as_index=False)
            .size()
            .rename(columns={'size': 'absolut_freq'})
        )
        
        if 'minutes' in grouped.columns:
            grouped['freq_per_hour'] = grouped['absolut_freq'] / (grouped['minutes'] / 60)
        else:

            grouped['freq_per_hour'] = grouped['absolut_freq']
        
        
        base = os.path.splitext(os.path.basename(in_path))[0]
        out_name = f"{base}_sections.csv"
        out_path = os.path.join(output_dir, out_name)
        grouped.to_csv(out_path, index=False, sep=output_sep)
        print(f"Salvestatud {len(grouped)} read → {out_path}")

In [49]:
if __name__ == "__main__":
    make_section_freq_tables(
        input_dir='.../particles_per_value',
        output_dir='.../particles_per_value/sections_partikkel'
    )

Salvestatud 81 read → .../particles_per_value/sections_partikkel/jah_sections.csv
Salvestatud 146 read → .../particles_per_value/sections_partikkel/okei_sections.csv
Salvestatud 481 read → .../particles_per_value/sections_partikkel/nagu_sections.csv
Salvestatud 71 read → .../particles_per_value/sections_partikkel/vot_sections.csv
Salvestatud 77 read → .../particles_per_value/sections_partikkel/aitäh_sections.csv
Salvestatud 183 read → .../particles_per_value/sections_partikkel/noh_sections.csv
Salvestatud 44 read → .../particles_per_value/sections_partikkel/oh_sections.csv
Salvestatud 36 read → .../particles_per_value/sections_partikkel/kurat_sections.csv
Salvestatud 211 read → .../particles_per_value/sections_partikkel/no_sections.csv


In [50]:
if __name__ == "__main__":
    save_tables_per_particle(
        csv_path='tabelid_kontrollimiseks/intensiivistajad_adverbidega.csv',
        column_name='inetnsiivistaja',
        out_dir='.../intensiivistajad_adverbidega_per_value',
        threshold=50 
    )

Salvestatud 378 read → .../intensiivistajad_adverbidega_per_value/hästi.csv
Salvestatud 962 read → .../intensiivistajad_adverbidega_per_value/kui.csv
Salvestatud 121 read → .../intensiivistajad_adverbidega_per_value/liiga.csv
Salvestatud 161 read → .../intensiivistajad_adverbidega_per_value/natuke.csv
Salvestatud 125 read → .../intensiivistajad_adverbidega_per_value/natukene.csv
Salvestatud 981 read → .../intensiivistajad_adverbidega_per_value/nii.csv
Salvestatud 248 read → .../intensiivistajad_adverbidega_per_value/palju.csv
Salvestatud 264 read → .../intensiivistajad_adverbidega_per_value/päris.csv
Salvestatud 118 read → .../intensiivistajad_adverbidega_per_value/täiesti.csv
Salvestatud 118 read → .../intensiivistajad_adverbidega_per_value/tõesti.csv
Salvestatud 911 read → .../intensiivistajad_adverbidega_per_value/väga.csv
Salvestatud 131 read → .../intensiivistajad_adverbidega_per_value/üldse.csv
Salvestatud 54 read → .../intensiivistajad_adverbidega_per_value/üsna.csv


In [51]:
if __name__ == "__main__":
    make_section_freq_tables(
        input_dir='.../intensiivistajad_adverbidega_per_value',
        output_dir='.../intensiivistajad_adverbidega_per_value/sections_intensiivistajad_adverbidega'
    )

Salvestatud 140 read → .../intensiivistajad_adverbidega_per_value/sections_intensiivistajad_adverbidega/päris_sections.csv
Salvestatud 76 read → .../intensiivistajad_adverbidega_per_value/sections_intensiivistajad_adverbidega/liiga_sections.csv
Salvestatud 36 read → .../intensiivistajad_adverbidega_per_value/sections_intensiivistajad_adverbidega/üsna_sections.csv
Salvestatud 91 read → .../intensiivistajad_adverbidega_per_value/sections_intensiivistajad_adverbidega/üldse_sections.csv
Salvestatud 77 read → .../intensiivistajad_adverbidega_per_value/sections_intensiivistajad_adverbidega/täiesti_sections.csv
Salvestatud 127 read → .../intensiivistajad_adverbidega_per_value/sections_intensiivistajad_adverbidega/palju_sections.csv
Salvestatud 272 read → .../intensiivistajad_adverbidega_per_value/sections_intensiivistajad_adverbidega/kui_sections.csv
Salvestatud 100 read → .../intensiivistajad_adverbidega_per_value/sections_intensiivistajad_adverbidega/natuke_sections.csv
Salvestatud 141 read

In [53]:
if __name__ == "__main__":
    save_tables_per_particle(
        csv_path='tabelid_kontrollimiseks/intensiivistajad_omdasusonadega.csv',
        column_name='inetnsiivistaja',
        out_dir='.../intensiivistajad_omadussonadega_per_value',
        threshold=50 
    )
    make_section_freq_tables(
        input_dir='.../intensiivistajad_omadussonadega_per_value',
        output_dir='.../intensiivistajad_omadussonadega_per_value/sections_intensiivistajad_omadussonadega'
    )

Salvestatud 453 read → .../intensiivistajad_omadussonadega_per_value/hästi.csv
Salvestatud 254 read → .../intensiivistajad_omadussonadega_per_value/kui.csv
Salvestatud 71 read → .../intensiivistajad_omadussonadega_per_value/liiga.csv
Salvestatud 51 read → .../intensiivistajad_omadussonadega_per_value/natuke.csv
Salvestatud 681 read → .../intensiivistajad_omadussonadega_per_value/nii.csv
Salvestatud 143 read → .../intensiivistajad_omadussonadega_per_value/palju.csv
Salvestatud 285 read → .../intensiivistajad_omadussonadega_per_value/päris.csv
Salvestatud 57 read → .../intensiivistajad_omadussonadega_per_value/suht.csv
Salvestatud 165 read → .../intensiivistajad_omadussonadega_per_value/täiesti.csv
Salvestatud 1764 read → .../intensiivistajad_omadussonadega_per_value/väga.csv
Salvestatud 67 read → .../intensiivistajad_omadussonadega_per_value/üldse.csv
Salvestatud 83 read → .../intensiivistajad_omadussonadega_per_value/üsna.csv
Salvestatud 154 read → .../intensiivistajad_omadussonadega_p

In [54]:
if __name__ == "__main__":
    save_tables_per_particle(
        csv_path='tabelid_kontrollimiseks/kp_markerid.csv',
        column_name='verb',
        out_dir='.../kp_markerid_per_value',
        threshold=50 
    )
    make_section_freq_tables(
        input_dir='.../kp_markerid_per_value',
        output_dir='.../kp_markerid_per_value/sections_kp_markerid'
    )

Salvestatud 1004 read → .../kp_markerid_per_value/arvan.csv
Salvestatud 97 read → .../kp_markerid_per_value/loodan.csv
Salvestatud 291 read → .../kp_markerid_per_value/tundub.csv
Salvestatud 142 read → .../kp_markerid_per_value/usun.csv
Salvestatud 930 read → .../kp_markerid_per_value/ütleme.csv
Salvestatud 228 read → .../kp_markerid_per_value/sections_kp_markerid/ütleme_sections.csv
Salvestatud 149 read → .../kp_markerid_per_value/sections_kp_markerid/tundub_sections.csv
Salvestatud 70 read → .../kp_markerid_per_value/sections_kp_markerid/loodan_sections.csv
Salvestatud 255 read → .../kp_markerid_per_value/sections_kp_markerid/arvan_sections.csv
Salvestatud 93 read → .../kp_markerid_per_value/sections_kp_markerid/usun_sections.csv
